# QAOA gridsearch

In [1]:
#Initialisation cell
from docplex.mp.model import Model
from qiskit.providers.ibmq.managed import IBMQJobManager

from qiskit import BasicAer, Aer
from qiskit.aqua.algorithms import VQE, ExactEigensolver,NumPyEigensolver,QAOA,NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import SPSA, COBYLA,ADAM, L_BFGS_B
from qiskit.circuit.library import RealAmplitudes #(uses CX entangling) 
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.circuit.library import TwoLocal
#from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance

from qiskit.optimization import QuadraticProgram
from qiskit.tools.visualization import plot_histogram
from qiskit.optimization.applications.ising import docplex
from qiskit.optimization.algorithms import MinimumEigenOptimizer,RecursiveMinimumEigenOptimizer

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
# set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

# useful additional packages 
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx
import pandas as pd
from   matplotlib import cm
from   matplotlib.ticker import LinearLocator, FormatStrFormatter
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

# importing Qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute
from qiskit.providers.ibmq      import least_busy
from qiskit.tools.monitor       import job_monitor
from qiskit.providers.aer import QasmSimulator

Important prelim functions

In [26]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [27]:
def qap_value(z, MatrixLoc, MatrixFlow):
    """Compute the TSP value of a solution.
    Args:
        z (list[int]): list of allocations
        MatrixLoc (numpy array): matrix of distances
        MatrixFlow (numpy array): matrix of flow
    Returns:
        float: value of the QAP
    """
    matrix_length = len(MatrixLoc)
    x = np.reshape(z, (matrix_length,matrix_length))
    
    total = 0
    for i in range(matrix_length):
        for j in range(matrix_length):
            for k in range(matrix_length):
                for p in range(matrix_length):
                        total += MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p]
    
    return total

Feasibility functions

In [28]:
def qap_feasible(x):
    """Check whether a solution is feasible or not.

    Args:
        x (numpy.ndarray) : binary string as numpy array.

    Returns:
        bool: feasible or not.
    """
    n = int(np.sqrt(len(x)))
    y = np.reshape(x, (n,n))
   
    for i in range(n):
        if sum(y[i, p] for p in range(n)) != 1:
            return False
    for p__ in range(n):
        if sum(y[i, p__] for i in range(n)) != 1:
            return False
    return True

In [29]:
def choose_best_feasible(eigenstates):
    """
    Input:
    eigenstates = dictionary
    
    Output:
    feasible binary 1D numpyarray
    probability of this answer
    
    """
    bestinarray = sorted(eigenstates.items(), key=lambda item: item[1])[::-1]
    feasible = False
    counter = 0
    total = sum(eigenstates.values())
    
    feasible=False
    while feasible==False and counter<len(bestinarray):
        #string to array
        bestasint = np.array([0])
        for i in bestinarray[counter][0]:
            bestasint = np.hstack((bestasint, int(i)))
        feasible = qap_feasible(bestasint[1:])
        frequency = bestinarray[counter][1]
        counter += 1
        
    if feasible == False:
        return feasible
    else:
        return bestasint[1:], frequency/total

## The QAOA function

In [30]:
def testing_quantum(ins, start, stop, step):
    """
    Input:
    ins: name of instance
    start
    stop
    step
    
    Output:
    outmatrix : [soln,result time,eigenvector,number of spsa trials]
    """
    
    #get matrix
    datamatrix = CSVtoNumpyArray(ins)
    MatrixLoc = datamatrix[0]
    MatrixFlow = datamatrix[1]
    n = len(MatrixLoc)

    # Create an instance of a model and variables.
    thename = "qap" + str(n)
    mdl = Model(name=thename)
    x = {(i,p): mdl.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}

    # Object function
    qap_func = mdl.sum(MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p] for i in range(n) for j in range(n) for p in range(n) for k in range(n))
    mdl.minimize(qap_func)

    # Constraints
    for i in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for p in range(n)) == 1)
    for p in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for i in range(n)) == 1)
    print(mdl.export_to_string())
    qubitOp_docplex, offset_docplex = docplex.get_operator(mdl) #potential error

    #QAOA
    machinesolutions = []
    rmachinesolutions = []
    seed = 10598
    mod = QuadraticProgram()
    mod.from_docplex(mdl)
    #print(mod.export_as_lp_string())

    for i in range(start,stop,step):
        for j in range(5,6,1):
            #spsa = L_BFGS_B(maxiter=i)
            spsa = SPSA(maxiter=i)
            sim = Aer.get_backend('qasm_simulator')
            #sim = QasmSimulator()
            backend_options = {"method": "matrix_product_state"}
            quantum_instance = QuantumInstance(backend=sim,seed_simulator=seed,
                                               seed_transpiler=seed, skip_qobj_validation = False, shots = 1,
                                               backend_options=backend_options)

            #qaoa = QAOA(qubitOp_docplex,optimizer=spsa, p=j, quantum_instance=quantum_instance,include_custom=True)
            #result = qaoa.run(backend=Aer.get_backend('qasm_simulator'))
            
            qaoa_mes = QAOA(quantum_instance=quantum_instance, include_custom=True, optimizer=spsa,p=j)
            exact_mes = NumPyMinimumEigensolver()
            qaoa = MinimumEigenOptimizer(qaoa_mes)
            exact = MinimumEigenOptimizer(exact_mes) 
            rqaoa = RecursiveMinimumEigenOptimizer(min_eigen_optimizer=qaoa, min_num_vars=1, min_num_vars_optimizer=exact)
            rqaoa_result = rqaoa.solve(mod)
            print(rqaoa_result)
            qaoa_result = qaoa.solve(mod)
            print(qaoa_result)
            #print('QAOA vector:', qaoa_result['eigenstate'])
            print('QAOA vector:', qaoa_result.x)
            print('RQAOA vector:', rqaoa_result.x)
            #print('QAOA time:', qaoa_result['optimizer_time'])
            #print('QAOA QAP objective:', qaoa_result['eigenvalue'] + offset_docplex)
            #x = choose_best_feasible(qaoa_result['eigenstate'])
            #print(x,i,j)
            x = qap_feasible(qaoa_result.x)
            print(x,i,j)
            rx = qap_feasible(rqaoa_result.x)
            print(rx,i,j)
            if x == False:
                soln = 'No solution' #note fails if false
            else:
                #soln = qap_value(x[0],MatrixFlow,MatrixLoc) #note fails if false
                soln = qap_value(x,MatrixFlow,MatrixLoc) #note fails if false
            if rx == False:
                rsoln = 'No solution' #note fails if false
            else:
                #soln = qap_value(x[0],MatrixFlow,MatrixLoc) #note fails if false
                rsoln = qap_value(rx,MatrixFlow,MatrixLoc) #note fails if false
            machinesolutions.append([soln,x,i])
            rmachinesolutions.append([rsoln,rx,i])
            #machinesolutions.append([soln,qaoa_result['optimizer_time'],x,i])
    print(machinesolutions)
    print(rmachinesolutions)
    return qaoa_result#machinesolutions
    

In [31]:
ins = "made3.csv"
start = 1000
stop = 2000
step = 1000
ans = testing_quantum(ins, start, stop, step)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 4704 x_0_0*x_1_1 + 3444 x_0_0*x_1_2 + 21280 x_0_0*x_2_1
      + 15580 x_0_0*x_2_2 + 4704 x_0_1*x_1_0 + 504 x_0_1*x_1_2
      + 21280 x_0_1*x_2_0 + 2280 x_0_1*x_2_2 + 3444 x_0_2*x_1_0
      + 504 x_0_2*x_1_1 + 15580 x_0_2*x_2_0 + 2280 x_0_2*x_2_1
      + 18368 x_1_0*x_2_1 + 13448 x_1_0*x_2_2 + 18368 x_1_1*x_2_0
      + 1968 x_1_1*x_2_2 + 13448 x_1_2*x_2_0 + 1968 x_1_2*x_2_1 ]/2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 = 1
 c2: x_1_0 + x_1_1 + x_1_2 = 1
 c3: x_2_0 + x_2_1 + x_2_2 = 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2
End



constraint c1 is infeasible due to substitution
constraint c2 is infeasible due to substitution
constraint c3 is infeasible due to substitution
constraint c4 is infeasible due to substitution


optimal function value: 1140.0
optimal value: [0. 1. 0. 0. 0. 0. 0. 1. 1.]
status: SUCCESS


constraint c0 is infeasible due to substitution
constraint c1 is infeasible due to substitution
constraint c2 is infeasible due to substitution
constraint c4 is infeasible due to substitution
constraint c5 is infeasible due to substitution


optimal function value: 24734.0
optimal value: [0. 1. 1. 1. 1. 1. 0. 1. 1.]
status: SUCCESS
QAOA vector: [0. 1. 1. 1. 1. 1. 0. 1. 1.]
RQAOA vector: [0. 1. 0. 0. 0. 0. 0. 1. 1.]
False 1000 5
False 1000 5
[['No solution', False, 1000]]
[['No solution', False, 1000]]


In [53]:
type(ans)

qiskit.optimization.algorithms.minimum_eigen_optimizer.MinimumEigenOptimizerResult

In [ ]:
problem = QuadraticProgram(qubitOp_docplex)
spsa = SPSA(maxiter=i)
sim = Aer.get_backend(machine)
#sim = QasmSimulator()
backend_options = {"method": "matrix_product_state"}
quantum_instance = QuantumInstance(backend=sim,seed_simulator=seed,
                                   seed_transpiler=seed, skip_qobj_validation = False, shots = 1,
                                   backend_options=backend_options)

qaoa = QAOA(qubitOp_docplex,optimizer=spsa, p=j, quantum_instance=quantum_instance,include_custom=True)

result = qaoa.run(backend=Aer.get_backend('qasm_simulator'))

In [11]:
import qiskit
help(qiskit.optimization.algorithms.minimum_eigen_optimizer.MinimumEigenOptimizerResult)

Help on class MinimumEigenOptimizerResult in module qiskit.optimization.algorithms.minimum_eigen_optimizer:

class MinimumEigenOptimizerResult(qiskit.optimization.algorithms.optimization_algorithm.OptimizationResult)
 |  MinimumEigenOptimizerResult(x: Union[List[float], numpy.ndarray], fval: float, variables: List[qiskit.optimization.problems.variable.Variable], samples: List[Tuple[str, float, float]], qubo_converter: qiskit.optimization.converters.quadratic_program_to_qubo.QuadraticProgramToQubo) -> None
 |  
 |  Minimum Eigen Optimizer Result.
 |  
 |  Method resolution order:
 |      MinimumEigenOptimizerResult
 |      qiskit.optimization.algorithms.optimization_algorithm.OptimizationResult
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, x: Union[List[float], numpy.ndarray], fval: float, variables: List[qiskit.optimization.problems.variable.Variable], samples: List[Tuple[str, float, float]], qubo_converter: qiskit.optimization.converters.quadratic_prog

In [14]:
theeeeans = ans
theeeeans

optimal function value: 24734.0
optimal value: [0. 1. 1. 1. 1. 1. 0. 1. 1.]
status: SUCCESS

In [4]:
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.16.2',
 'qiskit-aer': '0.7.3',
 'qiskit-ignis': '0.5.1',
 'qiskit-ibmq-provider': '0.11.1',
 'qiskit-aqua': '0.8.1',
 'qiskit': '0.23.3'}

In [7]:
from qiskit import IBMQ
IBMQ.load_account()
IBMQ.providers()
provider = IBMQ.get_provider(hub='ibm-q-wits',project='physics')
provider.backends()
provider.backend.jobs(backend_name=ibmq_boeblingen)
job.properties()

C:\Users\Admin\Anaconda3\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


AttributeError: 'AccountProvider' object has no attribute 'backend'

In [8]:
from qiskit.test.mock.backends import FakeBoeblingen
backend=FakeBoeblingen()
print(backend.properties().to_dict())

{'backend_name': 'alt_boeblingen', 'backend_version': '1.1.2', 'last_update_date': datetime.datetime(2020, 7, 16, 9, 31, 10, tzinfo=tzutc()), 'qubits': [[{'date': '2020-07-16T08:23:39+00:00', 'name': 'T1', 'unit': 'µs', 'value': 73.93949850765505}, {'date': '2020-07-16T08:24:23+00:00', 'name': 'T2', 'unit': 'µs', 'value': 123.86451996423287}, {'date': '2020-07-16T09:31:10+00:00', 'name': 'frequency', 'unit': 'GHz', 'value': 5.046628190712466}, {'date': '2020-07-16T08:23:05+00:00', 'name': 'readout_error', 'unit': '', 'value': 0.026666666666666616}, {'date': '2020-07-16T08:23:05+00:00', 'name': 'prob_meas0_prep1', 'unit': '', 'value': 0.046666666666666634}, {'date': '2020-07-16T08:23:05+00:00', 'name': 'prob_meas1_prep0', 'unit': '', 'value': 0.006666666666666667}], [{'date': '2020-07-16T08:23:39+00:00', 'name': 'T1', 'unit': 'µs', 'value': 35.20334130966316}, {'date': '2020-07-16T08:25:01+00:00', 'name': 'T2', 'unit': 'µs', 'value': 75.89467723515469}, {'date': '2020-07-16T09:31:10+00: